In [1]:
import csv
import os
import time
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

timestamp = datetime.now().strftime("%d-%m-%y_%H-%M-%S")

class Report:
    header_csv = [
        "exec_time",
        "acc",
        "sen",
        "spe",
        "segment_size",
        "quantity_bins",
        "quantity_segments",
        "range_hist",
        "layers",
    ]
    filename_csv = f"results_{timestamp}.csv"

    def __init__(self):
        self.start = time.time()
        self.end = None
        self.report_folds = []
        self.report_resume = None

    def register_report_fold(self, y, y_pred):
        report = classification_report(
            y,
            y_pred,
            labels=None,
            target_names=None,
            sample_weight=None,
            digits=2,
            output_dict=True,
            zero_division="warn",
        )
        report_fold = [
            report["accuracy"],
            report["1.0"]["recall"],
            report["0.0"]["recall"],
        ]
        self.report_folds.append(report_fold)

    def register_report_resume(self):
        acc_resume = np.array([r[0] for r in self.report_folds])
        sen_resume = np.array([r[1] for r in self.report_folds])
        spe_resume = np.array([r[2] for r in self.report_folds])
        label_acc_resume = f"{acc_resume.mean().round(4)}+-{acc_resume.std().round(4)}"
        label_sen_resume = f"{sen_resume.mean().round(4)}+-{sen_resume.std().round(4)}"
        label_spe_resume = f"{spe_resume.mean().round(4)}+-{spe_resume.std().round(4)}"
        self.report_resume = [label_acc_resume, label_sen_resume, label_spe_resume]

    def persist_report_resume(
        self, segment_size, quantity_bins, quantity_segments, range_hist, model
    ):
        str_model = str(model).replace("\n", "").replace("              ", "")
        str_model = str_model[92:100]

        row_data = [
            time.strftime("%H:%M:%S", time.gmtime(self.end - self.start)),
            self.report_resume[0],
            self.report_resume[1],
            self.report_resume[2],
            segment_size,
            quantity_bins,
            quantity_segments,
            range_hist,
            str_model,
        ]

        if not os.path.exists(self.filename_csv):
            with open(self.filename_csv, "w", newline="\n") as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=";")
                csvwriter.writerow(self.header_csv)
        else:
            with open(self.filename_csv, "r", newline="\n") as csvfile:
                csvreader = csv.reader(csvfile, delimiter=";")
                if not any(row for row in csvreader):
                    with open(self.filename_csv, "w", newline="\n") as csvfile:
                        csvwriter = csv.writer(csvfile, delimiter=";")
                        csvwriter.writerow(self.header_csv)

        with open(self.filename_csv, "a", newline="\n") as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=";")
            csvwriter.writerow(row_data)

    def get_df_results(self):
        report_full = self.report_folds[:]
        report_full.append(self.report_resume)
        indexes = list(range(1, 11))
        indexes.append("Average")
        return pd.DataFrame(
            report_full,
            columns=["Accuracy", "Sensibility", "Specificity"],
            index=indexes,
        )

    def print_report(self):
        self.end = time.time()
        self.register_report_resume()
        return self.get_df_results()